In [56]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import pickle
import mgzip
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegressionCV
import pyreadr
from sklearn.metrics import f1_score

tqdm.pandas()

### Choose file we want to predict

In [12]:
with mgzip.open('../data/predict_bert_topic.mgzip', 'rb') as handle:
    predict = pickle.load(handle)
#predict.to_csv('../data/predict_bert_topic.csv')

In [79]:
X = predict
X = X.dropna()
X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
X.drop('newspaper', inplace=True, axis=1)

/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/3869162192.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/3869162192.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/3869162192.py:3: SettingWithCopyWarning: 
A value is tr

#### Continuous labels

In [80]:
y = X['bert_score']
X.drop(['bert_score', 'bert_label'], axis=1, inplace=True)

/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/2893972162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(['bert_score', 'bert_label'], axis=1, inplace=True)


In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

##### Create baseline value using linear regression

In [19]:
regr = LinearRegression()
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.06330454354398718

In [20]:
y_pred = regr.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 0.17404903048763853


R2 is 6%, MAE is 0.174

##### Try ensemble random forest regressor
Grid search for parameter tuning, cross validation: 5

In [94]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10 , cv = 3, verbose = 3, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,y_train)

rf_random.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 3/3] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200;, score=0.090 total time=  22.5s
[CV 1/3] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1400;, score=0.076 total time= 2.6min


/opt/homebrew/Caskroom/miniforge/base/envs/NLP_thesis_new/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 2/3] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200;, score=0.079 total time=  23.0s
[CV 2/3] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1400;, score=0.080 total time= 2.6min
[CV 1/3] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200;, score=0.070 total time=  22.1s
[CV 3/3] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=600;, score=-0.598 total time=23.7min
[CV 2/3] END bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1200;, score=0.077 total time=27.6min
[CV 1/3] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600;, score=0.075 total time= 1.6min
[CV 1/3] END bootstrap=False, max_depth=80, max_features=auto, min_sample

{'n_estimators': 600,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}

In [95]:
#regr = RandomForestRegressor(verbose=1, n_jobs=-1)
regr = RandomForestRegressor(verbose = 1, n_jobs = -1, n_estimators = 600, min_samples_split = 10, min_samples_leaf = 4, max_features = 'sqrt', max_depth = 90, bootstrap = False)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:   18.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 600 out of 600 | elapsed:    0.2s finished


0.09278917149305532

In [96]:
y_pred = regr.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 0.16896458088878005


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 600 out of 600 | elapsed:    0.2s finished


R2 is 9.3%, MAE is 0.169

##### Try neural network
Rescaling of all features to 0-1 for better compatibility

In [24]:
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.fit_transform(X_test)

In [31]:
regr = MLPRegressor(max_iter=2000, verbose=2)

In [44]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,), (119,119,62)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05, 0.001],
    'learning_rate_init': [0.0001,0.001],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
clf = GridSearchCV(regr, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train_norm, y_train)

In [46]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'adam'}
-0.056 (+/-0.025) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.0001, 'solver': 'sgd'}
0.048 (+/-0.015) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.0001, 'solver': 'adam'}
0.014 (+/-0.009) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'sgd'}
0.046 (+/-0.040) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'adam'}
-0.047 (+/-0.029) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'lear

In [49]:
regr = MLPRegressor(max_iter=2000, verbose=2, activation='relu', alpha=0.05, hidden_layer_sizes=(50,100,50), learning_rate='constant', solver='adam', learning_rate_init=0.001)
regr.fit(X_train_norm, y_train)
regr.score(X_test_norm, y_test)

Iteration 1, loss = 0.05880228
Iteration 2, loss = 0.04845423
Iteration 3, loss = 0.04545185
Iteration 4, loss = 0.04409507
Iteration 5, loss = 0.04328532
Iteration 6, loss = 0.04246440
Iteration 7, loss = 0.04124994
Iteration 8, loss = 0.04100071
Iteration 9, loss = 0.04031971
Iteration 10, loss = 0.03969308
Iteration 11, loss = 0.03953992
Iteration 12, loss = 0.03904640
Iteration 13, loss = 0.03864370
Iteration 14, loss = 0.03834231
Iteration 15, loss = 0.03826871
Iteration 16, loss = 0.03764258
Iteration 17, loss = 0.03731404
Iteration 18, loss = 0.03745823
Iteration 19, loss = 0.03725181
Iteration 20, loss = 0.03685206
Iteration 21, loss = 0.03643515
Iteration 22, loss = 0.03656196
Iteration 23, loss = 0.03620201
Iteration 24, loss = 0.03639028
Iteration 25, loss = 0.03608182
Iteration 26, loss = 0.03584800
Iteration 27, loss = 0.03566277
Iteration 28, loss = 0.03555200
Iteration 29, loss = 0.03529609
Iteration 30, loss = 0.03550666
Iteration 31, loss = 0.03535698
Iteration 32, los

0.06472387378707056

In [50]:
y_pred = regr.predict(X_test_norm)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 0.1726337614682266


R2 is 6%, MAE is 0.172

#### Categorical labels

In [97]:
X = predict
X = X.dropna()
X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
X.drop('newspaper', inplace=True, axis=1)
y = X['bert_label']
X.drop(['bert_score', 'bert_label'], axis=1, inplace=True)

/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/526324045.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/526324045.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/526324045.py:3: SettingWithCopyWarning: 
A value is tryin

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

##### Create baseline value using logistic regression

In [99]:
regr = LogisticRegressionCV()
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.5686298976790617

In [100]:
f1_score(y_test, regr.predict(X_test), average='macro')

0.24166732956805345

Accuracy is 57%, F1 macro is 24.1%

##### Try ensemble random forest classifier

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
criterion = ['gini', 'entropy']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
max_features = ['sqrt', 'log2', None]
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'criterion': criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10 , cv = 3, verbose = 3, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,y_train)

rf_random.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [102]:
regr = RandomForestClassifier(verbose = 1, n_jobs = -1, n_estimators = 200, min_samples_split = 10, min_samples_leaf = 4, max_features = None, max_depth = 80, criterion = 'entropy', bootstrap=True)
regr.fit(X_train,y_train)
regr.score(X_test,y_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.3s


[CV 3/3] END bootstrap=False, criterion=entropy, max_depth=110, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=1200;, score=0.651 total time= 3.0min
[CV 1/3] END bootstrap=False, criterion=gini, max_depth=60, max_features=None, min_samples_leaf=2, min_samples_split=2, n_estimators=1000;, score=0.556 total time=24.0min


[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   58.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.1s finished


0.6488644871474919

In [105]:
f1_score(y_test, regr.predict(X_test), average='macro')

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.1s finished


0.43368988575543943

Accuracy is 64.8%, F1 macro is 43.3%

##### Try neural network

In [67]:
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.fit_transform(X_test)

In [68]:
regr = MLPClassifier(verbose = 1, max_iter=3000)

In [69]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,), (119,119,62)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05, 0.001],
    'learning_rate_init': [0.0001,0.001],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
clf = GridSearchCV(regr, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train_norm, y_train)

In [71]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (119, 119, 62), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'sgd'}
0.619 (+/-0.020) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.0001, 'solver': 'sgd'}
0.632 (+/-0.017) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.0001, 'solver': 'adam'}
0.630 (+/-0.015) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'sgd'}
0.592 (+/-0.041) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'adam'}
0.609 (+/-0.016) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'learni

In [73]:
regr = MLPClassifier(max_iter=2000, verbose=2, activation='relu', alpha=0.05, hidden_layer_sizes=(119,119,62), learning_rate='constant', solver='sgd', learning_rate_init=0.001)
regr.fit(X_train_norm, y_train)
regr.score(X_test_norm, y_test)

Iteration 1, loss = 0.84886440
Iteration 2, loss = 0.79892494
Iteration 3, loss = 0.79533719
Iteration 4, loss = 0.79222650
Iteration 5, loss = 0.78895374
Iteration 6, loss = 0.78571603
Iteration 7, loss = 0.78265068
Iteration 8, loss = 0.77980968
Iteration 9, loss = 0.77753542
Iteration 10, loss = 0.77470230
Iteration 11, loss = 0.77236425
Iteration 12, loss = 0.77025055
Iteration 13, loss = 0.76851908
Iteration 14, loss = 0.76667344
Iteration 15, loss = 0.76542007
Iteration 16, loss = 0.76378636
Iteration 17, loss = 0.76258822
Iteration 18, loss = 0.76131407
Iteration 19, loss = 0.76018297
Iteration 20, loss = 0.75916244
Iteration 21, loss = 0.75784883
Iteration 22, loss = 0.75737909
Iteration 23, loss = 0.75652252
Iteration 24, loss = 0.75558592
Iteration 25, loss = 0.75483001
Iteration 26, loss = 0.75392748
Iteration 27, loss = 0.75307048
Iteration 28, loss = 0.75218877
Iteration 29, loss = 0.75170372
Iteration 30, loss = 0.75035988
Iteration 31, loss = 0.75033041
Iteration 32, los

0.6395058647367108

In [76]:
f1_score(y_test, regr.predict(X_test_norm), average='macro')

0.39072754266319415

Accuracy is 63.9%, F1 macro is 39.1%